In [69]:
import pandas as pd
import numpy as np
import os


import Finalized_pipeline

import seaborn as sns
import matplotlib.pyplot as plt

In [70]:
main_datasets = ["BACE_class", "BACE_regre", "ROR_class", "ROR_regre"]
model_name_shorts = {"DecisionTreeRegressor": "dt", "DecisionTreeClassifier": "dt", 
                    "RandomForestRegressor": "rf", "RandomForestClassifier": "rf", 
                    "LinearRegression": "lr", "LogisticRegression": "lr", 
                    "MLPRegressor": "nn", "MLPClassifier": "nn",  
                    "GradientBoostingRegressor": "gb", "GradientBoostingClassifier": "gb",
                    "XGBRegressor": "xg", "XGBClassifier": "xg", 
                    "SVR": "sv", "SVC": "sv"}
    
top_models = 10


In [71]:
for dataset in main_datasets:
    regression = False
    if "regre" in dataset:
        regression = True


    df = pd.read_csv(r"Merged runs/" + dataset + ".csv")
    
    setups = {"Descriptors": [], "Fingerprints": [], "DescFing": []}

    for row in df.head(top_models)[['hyperparams', 'preprocessing', 'model']].iterrows():
        #print(row)
        params = list(row[1])[0].split(", '")
        params[0] = params[0][2:]
        params[-1] = params[-1][:-1]
        #print(params)

        preprocessing = list(row[1])[1]
        model = list(row[1])[2]
        #print(preprocessing)

        clean_params = {}
        
        for param in params:
            name = param.split("': ")[0]
            val = param.split("': ")[1]
            if val[0] == "'":
                val = val[1:-1]
            elif val[0] == "(":
                if val == "(50, 50)": val = (50, 50)
                if val == "(100, )" or val == "(100,)": val = (100, )
                if val == "(50, )" or val == "(50,)": val = (50, )
            elif val == "None":
                val = None
            else:
                val = float(val)
            clean_params[name] = val
        #print(model)
        #print(preprocessing)
        #print(clean_params)
        setups[preprocessing].append([model, clean_params])

    for key in setups.keys():
        setups[key] = sorted(setups[key], key=lambda x:x[0])
    print(setups)

    returned_df = pd.DataFrame(columns=df.columns)
    print(returned_df)

    for key in setups.keys():
        if key == "Descriptors":
            calculate_descriptors = True
            calculate_fingerprints = False
        if name == "DescFing":
            calculate_descriptors = True
            calculate_fingerprints = True
        if key == "Fingerprints":
            calculate_descriptors = False
            calculate_fingerprints = True

            
        if "BACE" in dataset:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_bace.csv")
        else:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_ROR_data_1.csv")
        
        if not "pIC50" in prepared_dataset.columns:
            prepared_dataset = Finalized_pipeline.calculate_pIC50(prepared_dataset, "target")
        
        if not regression:
            prepared_dataset = Finalized_pipeline.calculate_classification_labels(prepared_dataset, "pIC50", threshold=7)
            target_column = "label"
        else:
            target_column = "pIC50"

        if "mol" in prepared_dataset.columns:
            prepared_dataset.rename(columns={"mol": "SMILES"}, inplace=True)

        if regression:
            runtype = "regression"
        else:
            runtype = "classification"
        
        prepared_dataset = Finalized_pipeline.calculate_features(prepared_dataset, calculate_descriptors=calculate_descriptors, 
                                                                 calculate_fingerprints=calculate_fingerprints, 
                                                    SMILES_column_name="SMILES", target_column_name=target_column, 
                                                    split_column_name="Split")

        for combination in setups[key]:
                            # model short name, parameters
            hyperparams = {model_name_shorts[combination[0]]: combination[1]}
            new_df = Finalized_pipeline.hyperparameter_search(prepared_dataset, hyperparams, return_df=True)

            returned_df = pd.concat([returned_df, new_df])

    returned_df.to_csv("RERUN_" + dataset + ".csv")
        
                


{'Descriptors': [['MLPClassifier', {'hidden_layer_sizes': (100,), 'activation': 'relu', 'alpha': 0.01, 'max_iter': 200.0, 'solver': 'adam'}], ['MLPClassifier', {'hidden_layer_sizes': (50,), 'activation': 'tanh', 'alpha': 0.0001, 'max_iter': 500.0, 'solver': 'adam'}], ['SVC', {'C': 0.01, 'kernel': 'poly', 'gamma': 0.05}], ['SVC', {'C': 1.0, 'kernel': 'rbf', 'gamma': 0.05}]], 'Fingerprints': [['GradientBoostingClassifier', {'max_depth': 3.0, 'loss': 'exponential', 'n_estimators': 200.0, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}], ['MLPClassifier', {'hidden_layer_sizes': (100,), 'activation': 'relu', 'alpha': 0.001, 'max_iter': 1000.0, 'solver': 'sgd'}]], 'DescFing': [['LogisticRegression', {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}], ['MLPClassifier', {'hidden_layer_sizes': (50, 50), 'activation': 'relu', 'alpha': 0.01, 'max_iter': 500.0, 'solver': 'sgd'}], ['MLPClassifier', {'hidden_layer_sizes': (100,), 'activation': 'tanh', 'alpha': 0.001, 'max_iter': 1000.0, 'solve


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
overflow encountered in reduce                    
100%|██████████| 1513/1513 [04:10<00:00,  6.04it/s]


TypeError: hyperparameter_search() got an unexpected keyword argument 'return_df'